# 1. 필요 데이터 불러오기

- '5개동격자(인구,상권,연면적,버스).geojson' : 광양읍, 광영동,중마동,태인동

=> 태인동 제외하기

In [14]:
import pandas as pd
import geopandas as gpd
df1 = gpd.read_file('5개동격자(인구_상권_연면적_버스).geojson')
df2 = gpd.read_file('광양시_중심점별_가장_가까운_정류장_거리.geojson')
df3 = gpd.read_file('격자중심과_가까운_건물_평균좌표와의_거리_데이터.geojson')

# 태인동 제거
df1 = df1[df1['EMD_KOR_NM'] != "태인동"]

In [2]:
df1 = df1.groupby('gid')[['상권','인구정보','연면적']].sum()
df1 = df1.reset_index()
df1

,gid,상권,인구정보,연면적
0,라라042638,0.0,0.0,0.0
1,라라042639,0.0,0.0,0.0
2,라라042640,0.0,0.0,0.0
3,라라042641,0.0,0.0,0.0
4,라라043626,0.0,0.0,0.0
...,...,...,...,...
7259,라라210625,0.0,0.0,0.0
7260,라라210626,0.0,0.0,0.0
7261,라라210627,0.0,0.0,0.0
7262,라라211622,0.0,0.0,0.0


## 1-1. 격자 중심과 가까운 버스정류장 거리 변수 추가

In [3]:
df2 = pd.merge(df1,df2,on='gid',how='left')
df2 = df2.iloc[:,[0,1,2,3,7,8]]
df2 = df2.rename(columns={'HubDist' : 'Busdistance'})
df2

,gid,상권,인구정보,연면적,Busdistance,geometry
0,라라042638,0.0,0.0,0.0,684.841909,"MULTILINESTRING ((127.54656 34.96953, 127.5513..."
1,라라042639,0.0,0.0,0.0,764.407219,"MULTILINESTRING ((127.54656 34.97043, 127.5513..."
2,라라042640,0.0,0.0,0.0,848.371929,"MULTILINESTRING ((127.54656 34.97133, 127.5513..."
3,라라042641,0.0,0.0,0.0,935.552263,"MULTILINESTRING ((127.54656 34.97223, 127.5513..."
4,라라043626,0.0,0.0,0.0,758.681938,"MULTILINESTRING ((127.54765 34.95871, 127.5513..."
...,...,...,...,...,...,...
7259,라라210625,0.0,0.0,0.0,571.839376,"MULTILINESTRING ((127.73056 34.95760, 127.7270..."
7260,라라210626,0.0,0.0,0.0,491.997425,"MULTILINESTRING ((127.73056 34.95850, 127.7270..."
7261,라라210627,0.0,0.0,0.0,420.985826,"MULTILINESTRING ((127.73057 34.95940, 127.7270..."
7262,라라211622,0.0,0.0,0.0,717.366925,"MULTILINESTRING ((127.73165 34.95489, 127.7254..."


## 1-2. 격자 중심과 가까운 건물평균좌표 거리 변수 추가

In [4]:
df3 = pd.merge(df2,df3,on='gid',how='left')
df3 = df3.iloc[:,[0,1,2,3,4,5,9]]
df3 = df3.rename(columns={'geometry_x':'geometry'})
df3 = df3.rename(columns={'상권':'상권개수'})
df3 = df3[['gid','상권개수','인구정보','연면적','Busdistance','HubDist','geometry']]
df3 = df3.fillna(0)

In [5]:
df3

,gid,상권개수,인구정보,연면적,Busdistance,HubDist,geometry
0,라라042638,0.0,0.0,0.0,684.841909,113.053346,"MULTILINESTRING ((127.54656 34.96953, 127.5513..."
1,라라042639,0.0,0.0,0.0,764.407219,164.761101,"MULTILINESTRING ((127.54656 34.97043, 127.5513..."
2,라라042640,0.0,0.0,0.0,848.371929,248.229660,"MULTILINESTRING ((127.54656 34.97133, 127.5513..."
3,라라042641,0.0,0.0,0.0,935.552263,302.961753,"MULTILINESTRING ((127.54656 34.97223, 127.5513..."
4,라라043626,0.0,0.0,0.0,758.681938,434.933763,"MULTILINESTRING ((127.54765 34.95871, 127.5513..."
...,...,...,...,...,...,...,...
7259,라라210625,0.0,0.0,0.0,571.839376,159.923829,"MULTILINESTRING ((127.73056 34.95760, 127.7270..."
7260,라라210626,0.0,0.0,0.0,491.997425,163.710699,"MULTILINESTRING ((127.73056 34.95850, 127.7270..."
7261,라라210627,0.0,0.0,0.0,420.985826,116.508521,"MULTILINESTRING ((127.73057 34.95940, 127.7270..."
7262,라라211622,0.0,0.0,0.0,717.366925,144.202821,"MULTILINESTRING ((127.73165 34.95489, 127.7254..."


# 2. 가중치를 이용한 trash_예측값 변수 추가

## 2-1. 0만 있는  raw 삭제

In [6]:
a = df3[(df3['상권개수']==0) & (df3['인구정보']==0) & (df3['연면적']==0)]
idx = a.index
idx
df4 = df3.drop(idx).reset_index(drop=True)

In [7]:
df4

,gid,상권개수,인구정보,연면적,Busdistance,HubDist,geometry
0,라라043638,0.0,0.0,990.00,625.302475,24.129417,"MULTILINESTRING ((127.54765 34.96953, 127.5513..."
1,라라044638,0.0,10.0,76.28,577.073587,16.700679,"MULTILINESTRING ((127.54875 34.96953, 127.5513..."
2,라라044639,0.0,0.0,96.94,669.569856,45.899115,"MULTILINESTRING ((127.54875 34.97043, 127.5513..."
3,라라045631,3.0,0.0,697.87,225.513382,12.743386,"MULTILINESTRING ((127.54984 34.96322, 127.5513..."
4,라라045634,2.0,0.0,396.00,186.727587,66.215321,"MULTILINESTRING ((127.54984 34.96592, 127.5513..."
...,...,...,...,...,...,...,...
1707,라라209627,0.0,0.0,233.64,350.922273,22.074343,"MULTILINESTRING ((127.72947 34.95940, 127.7270..."
1708,라라209628,14.0,0.0,182.66,280.036873,31.160570,"MULTILINESTRING ((127.72947 34.96030, 127.7270..."
1709,라라209629,6.0,0.0,1033.76,231.950987,45.414794,"MULTILINESTRING ((127.72948 34.96121, 127.7270..."
1710,라라209630,0.0,0.0,278.44,222.009614,102.142908,"MULTILINESTRING ((127.72948 34.96211, 127.7270..."


## 2-2. 가중치 값 곱해서 trash 예측값 추가

|변수|가중치|
|------|---|
|상권|-0.320905|
|인구정보|2.36554888|
|연면적|0.04822848|
|Busdistance|0.09531933|
|HubDist|-0.62530756|

|변수|가중치|
|------|---|
|y 절편|-0.59462187|

In [8]:
a = [-0.320905  ,  2.36554888,  0.04822848,  0.09531933, -0.62530756]
y = -0.59462187

In [9]:
df4['trash_pd'] = (df4['상권개수']*a[0])+(df4['인구정보']*a[1])+(df4['연면적']*a[2])+(df4['Busdistance']*a[3])+(df4['HubDist']*a[4]) + y

In [10]:
df4 = df4[['gid','상권개수','인구정보','연면적','Busdistance','HubDist','trash_pd','geometry']]
df4

,gid,상권개수,인구정보,연면적,Busdistance,HubDist,trash_pd,geometry
0,라라043638,0.0,0.0,990.00,625.302475,24.129417,91.666680,"MULTILINESTRING ((127.54765 34.96953, 127.5513..."
1,라라044638,0.0,10.0,76.28,577.073587,16.700679,71.302942,"MULTILINESTRING ((127.54875 34.96953, 127.5513..."
2,라라044639,0.0,0.0,96.94,669.569856,45.899115,39.202534,"MULTILINESTRING ((127.54875 34.97043, 127.5513..."
3,라라045631,3.0,0.0,697.87,225.513382,12.743386,45.627122,"MULTILINESTRING ((127.54984 34.96322, 127.5513..."
4,라라045634,2.0,0.0,396.00,186.727587,66.215321,-5.744146,"MULTILINESTRING ((127.54984 34.96592, 127.5513..."
...,...,...,...,...,...,...,...,...
1707,라라209627,0.0,0.0,233.64,350.922273,22.074343,30.319902,"MULTILINESTRING ((127.72947 34.95940, 127.7270..."
1708,라라209628,14.0,0.0,182.66,280.036873,31.160570,10.930109,"MULTILINESTRING ((127.72947 34.96030, 127.7270..."
1709,라라209629,6.0,0.0,1033.76,231.950987,45.414794,41.047820,"MULTILINESTRING ((127.72948 34.96121, 127.7270..."
1710,라라209630,0.0,0.0,278.44,222.009614,102.142908,-29.874809,"MULTILINESTRING ((127.72948 34.96211, 127.7270..."


## 2-3. 쓰레기 배출량의 음수값은 없으므로 필터링

In [11]:
df4 = df4[df4['trash_pd']>0]
df4

,gid,상권개수,인구정보,연면적,Busdistance,HubDist,trash_pd,geometry
0,라라043638,0.0,0.0,990.00,625.302475,24.129417,91.666680,"MULTILINESTRING ((127.54765 34.96953, 127.5513..."
1,라라044638,0.0,10.0,76.28,577.073587,16.700679,71.302942,"MULTILINESTRING ((127.54875 34.96953, 127.5513..."
2,라라044639,0.0,0.0,96.94,669.569856,45.899115,39.202534,"MULTILINESTRING ((127.54875 34.97043, 127.5513..."
3,라라045631,3.0,0.0,697.87,225.513382,12.743386,45.627122,"MULTILINESTRING ((127.54984 34.96322, 127.5513..."
5,라라045637,0.0,11.0,120.39,447.031785,21.098550,60.650330,"MULTILINESTRING ((127.54984 34.96863, 127.5513..."
...,...,...,...,...,...,...,...,...
1706,라라209622,3.0,0.0,55.06,567.388961,27.516932,37.974713,"MULTILINESTRING ((127.72946 34.95489, 127.7251..."
1707,라라209627,0.0,0.0,233.64,350.922273,22.074343,30.319902,"MULTILINESTRING ((127.72947 34.95940, 127.7270..."
1708,라라209628,14.0,0.0,182.66,280.036873,31.160570,10.930109,"MULTILINESTRING ((127.72947 34.96030, 127.7270..."
1709,라라209629,6.0,0.0,1033.76,231.950987,45.414794,41.047820,"MULTILINESTRING ((127.72948 34.96121, 127.7270..."


# 3. 최종 데이터 프레임 저장

## 3-1. geometry 변수 -> line에서 polygon으로 변환

In [15]:
geometry = gpd.read_file('5개동격자(인구_상권_연면적_버스).geojson')
geometry = geometry.iloc[:,[0,6]]
geometry

,gid,geometry
0,라라042638,"MULTIPOLYGON (((127.54668 34.96998, 127.54711 ..."
1,라라042639,"MULTIPOLYGON (((127.54628 34.97088, 127.54711 ..."
2,라라042640,"MULTIPOLYGON (((127.54681 34.97178, 127.54711 ..."
3,라라042641,"MULTIPOLYGON (((127.54681 34.97178, 127.54688 ..."
4,라라042641,"MULTIPOLYGON (((127.54711 34.97205, 127.54711 ..."
...,...,...
8171,라라245618,"MULTIPOLYGON (((127.76833 34.95076, 127.76833 ..."
8172,라라245619,"MULTIPOLYGON (((127.76833 34.95166, 127.76833 ..."
8173,라라245620,"MULTIPOLYGON (((127.76833 34.95256, 127.76834 ..."
8174,라라245621,"MULTIPOLYGON (((127.76834 34.95346, 127.76834 ..."


In [12]:
geometry = gpd.read_file('5개동격자(인구_상권_연면적_버스).geojson')
geometry = geometry.iloc[:,[0,6]]
df_final = pd.merge(df4.iloc[:,[0,1,2,3,4,5,6]],geometry,on='gid',how='left')
df_final

,gid,상권개수,인구정보,연면적,Busdistance,HubDist,trash_pd,geometry
0,라라043638,0.0,0.0,990.00,625.302475,24.129417,91.666680,"MULTIPOLYGON (((127.54710 34.96942, 127.54711 ..."
1,라라044638,0.0,10.0,76.28,577.073587,16.700679,71.302942,"MULTIPOLYGON (((127.54820 34.96908, 127.54820 ..."
2,라라044639,0.0,0.0,96.94,669.569856,45.899115,39.202534,"MULTIPOLYGON (((127.54820 34.96998, 127.54820 ..."
3,라라045631,3.0,0.0,697.87,225.513382,12.743386,45.627122,"MULTIPOLYGON (((127.54929 34.96277, 127.54929 ..."
4,라라045637,0.0,11.0,120.39,447.031785,21.098550,60.650330,"MULTIPOLYGON (((127.54930 34.96818, 127.54930 ..."
...,...,...,...,...,...,...,...,...
1610,라라209627,0.0,0.0,233.64,350.922273,22.074343,30.319902,"MULTIPOLYGON (((127.72892 34.95895, 127.72892 ..."
1611,라라209627,0.0,0.0,233.64,350.922273,22.074343,30.319902,"MULTIPOLYGON (((127.73002 34.95933, 127.73002 ..."
1612,라라209628,14.0,0.0,182.66,280.036873,31.160570,10.930109,"MULTIPOLYGON (((127.72892 34.95985, 127.72893 ..."
1613,라라209629,6.0,0.0,1033.76,231.950987,45.414794,41.047820,"MULTIPOLYGON (((127.72893 34.96076, 127.72893 ..."


# 4. 데이터 저장

In [13]:
df_final.to_csv('금호동_제외_최종_데이터.csv')